In [1]:
import os
go_up_n_directories = lambda path, n: os.path.abspath(
    os.path.join(*([os.path.dirname(path)] + [".."] * n))
)
try:
    suda=suda # will give an error if this cell has not run before
except:
    os.chdir(go_up_n_directories(os.getcwd(), 3))  # run once (otherwise restart kernel)
    suda=True

In [4]:
# DEPENDENCIES
# Python native
import os
import pickle
import pprint
import random
import functools
import json
import time
from copy import copy
from datetime import datetime
from statistics import median as median
from sys import platform
from typing import Any, Callable

# Data handling
import numpy as np
import ocpa.algo.predictive_monitoring.factory as feature_factory

# PyG
import torch

# PyTorch TensorBoard support
import torch.utils.tensorboard

# Object centric process mining
from ocpa.algo.predictive_monitoring.obj import Feature_Storage as FeatureStorage

# # Simple machine learning models, procedure tools, and evaluation metrics
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.tensorboard.writer import SummaryWriter
from torch_geometric.loader import DataLoader
import torch_geometric.transforms as T
from tqdm import tqdm
from torch import tensor

# Custom imports
from experiments.efg import EFG
from experiments.efg_sg import EFG_SG
from utilities import (
    data_utils,
    evaluation_utils,
    experiment_utils,
    training_utils,
)
import utilities.torch_utils

# from importing_ocel import build_feature_storage, load_ocel, pickle_feature_storage
from models.definitions.geometric_models import (
    AGNN_EFG,
    AdamsGCN,
    GraphModel,
    HigherOrderGNN,
    SimpleGNN_EFG,
)
import torch_geometric.nn as pygnn
import torch.nn.functional as F
import torch.optim as O
import torch.nn as nn

# Print system info
utilities.torch_utils.print_system_info()

# Setup
bpi17_efg_config = {
    "model_output_path": "models/BPI17/efg",
    "STORAGE_PATH": "data/BPI17/feature_encodings/EFG/efg",
    "SPLIT_FEATURE_STORAGE_FILE": "BPI_split_[C2_P2_P3_P5_O3_Action_EventOrigin_OrgResource].fs",
    "TARGET_LABEL": (feature_factory.EVENT_REMAINING_TIME, ()),
    "regression_task": True,
    "graph_level_prediction": False,
    "dataset_class": EFG,
    "features_dtype": torch.float32,
    "target_dtype": torch.float32,
    "SUBGRAPH_SIZE": 4,
    "BATCH_SIZE": 64,
    "RANDOM_SEED": 42,
    "EPOCHS": 30,
    "early_stopping": 4,
    "hidden_dim": 64,
    "optimizer": O.Adam,
    "optimizer_settings": {
        "lr": 0.001,
        "betas": (0.9, 0.999),
        "eps": 1e-08,
        "weight_decay": 0,
        "amsgrad": False,
    },
    "loss_fn": torch.nn.L1Loss(),
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    "verbose": True,
    "track_time": True,
    "skip_cache": False,
    "squeeze": True,
}

# ADAPTATIONS
bpi17_efg_config["skip_cache"] = True

CPU: Intel(R) Core(TM) i5-7500 CPU @ 3.40GHz (4x)
Total CPU memory: 46.93GB
Available CPU memory: 32.55GB
GPU: NVIDIA GeForce GTX 960
Total GPU memory: 4096.0MB
Available GPU memory: 3128.0MB
Platform: Linux-6.2.0-26-generic-x86_64-with-glibc2.35


In [5]:
# Get data and dataloaders
ds_train, ds_val, ds_test = data_utils.load_datasets(
    dataset_class=bpi17_efg_config["dataset_class"],
    storage_path=bpi17_efg_config["STORAGE_PATH"],
    split_feature_storage_file=bpi17_efg_config["SPLIT_FEATURE_STORAGE_FILE"],
    target_label=bpi17_efg_config["TARGET_LABEL"],
    graph_level_target=bpi17_efg_config["graph_level_prediction"],
    features_dtype=bpi17_efg_config["features_dtype"],
    target_dtype=bpi17_efg_config["target_dtype"],
    subgraph_size=bpi17_efg_config["SUBGRAPH_SIZE"],
    train=True,
    val=True,
    test=True,
    skip_cache=bpi17_efg_config["skip_cache"],
)
train_loader, val_loader, test_loader = data_utils.prepare_dataloaders(
    batch_size=bpi17_efg_config["BATCH_SIZE"],
    ds_train=ds_train,
    ds_val=ds_val,
    ds_test=ds_test,
    num_workers=3,
    seed_worker=functools.partial(
        utilities.torch_utils.seed_worker, state=bpi17_efg_config["RANDOM_SEED"]
    ),
    generator=torch.Generator().manual_seed(bpi17_efg_config["RANDOM_SEED"]),
)

Processing...
/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
17645it [00:06, 2836.73it/s]
Done!
Processing...
4411it [00:01, 2727.10it/s]
Done!
Processing...
9453it [00:03, 2519.64it/s]
Done!


In [29]:
# # MODEL INITIALIZATION
class HigherOrderGNN(GraphModel):
    def __init__(
        self,
        hidden_channels: int = 48,
        out_channels: int = 1,
        no_mp_layers: int = 4,
        regression_target: bool = True,
        graph_level_prediction: bool = True,
    ):
        super().__init__()
        self.bn1 = pygnn.BatchNorm(hidden_channels)
        self.convs = nn.ModuleList()
        self.convs.append(pygnn.GraphConv(-1, hidden_channels))
        self.acts = nn.ModuleList()
        for i in range(no_mp_layers - 1):
            self.convs.append(pygnn.GraphConv(hidden_channels, hidden_channels))
            self.acts.append(nn.PReLU())
        self.lin_out = pygnn.Linear(-1, out_channels)

    def forward(self, x, edge_index, batch):
        for conv, act in zip(self.convs, self.acts):
            x = conv(x, edge_index)
            x = act(x)
        x = self.lin_out(x)
        return x


model = HigherOrderGNN(16, 1)
# # pretrained_state_dict = torch.load("models/runs/GraphConvNet_20230718_13h59m/state_dict_epoch6.pt")
# # model.load_state_dict(pretrained_state_dict)

# Print summary of data and model
if bpi17_efg_config["verbose"]:
    print(model)
    with torch.no_grad():  # Initialize lazy modules, s.t. we can count its parameters.
        batch = next(iter(train_loader))
        batch.to(bpi17_efg_config["device"])
        model.to(bpi17_efg_config["device"])
        out = model(batch.x.float(), batch.edge_index, batch.batch)
        print(f"Number of parameters: {utilities.torch_utils.count_parameters(model)}")

HigherOrderGNN(
  (bn1): BatchNorm(16)
  (convs): ModuleList(
    (0): GraphConv(-1, 16)
    (1): GraphConv(16, 16)
    (2): GraphConv(16, 16)
    (3): GraphConv(16, 16)
  )
  (acts): ModuleList(
    (0): SELU()
    (1): SELU()
    (2): SELU()
  )
  (lin_out): Linear(-1, 1, bias=True)
)
Number of parameters: 2513


In [30]:
bpi17_efg_config["model_output_path"] = "models/BPI17/efg/no_subgraph_sampling"

experiment_utils.run_efg_experiment_configuration(
    model_class=HigherOrderGNN,
    lr=0.01,
    hidden_dim=32,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    efg_config=bpi17_efg_config,
)


lr=0.01, hidden_dim=32:
Training started, progress available in Tensorboard
EPOCH 0:


276it [00:03, 87.19it/s]


Epoch loss -> train: 0 valid: 0.5377864241600037
EPOCH 1:


276it [00:03, 87.39it/s]


Epoch loss -> train: 0 valid: 0.5356507301330566
EPOCH 2:


...

Epoch loss -> train: 0 valid: 0.535510778427124
EPOCH 13:


276it [00:02, 107.35it/s]


Epoch loss -> train: 0 valid: 0.5271446108818054
Early stopping after 14 epochs.


100%|██████████| 148/148 [00:16<00:00,  8.84it/s]

lr=0.01, hidden_dim=32:
    8097 parameters
    0:00:57.702201 H:m:s
    0.5073


### Final hyperparameter tuning

In [12]:
lr_range = [0.01, 0.001]
hidden_dim_range = [8, 16, 24, 32, 48, 64, 128, 256]
for lr in lr_range:
    for hidden_dim in hidden_dim_range:
        experiment_utils.run_efg_experiment_configuration(
            model_class=HigherOrderGNN_EFG,
            lr=lr,
            hidden_dim=hidden_dim,
            track_time=True,
            verbose=False,
        )

Early stopping after 18 epochs.
lr=0.01, hidden_dim=8:
    587 parameters
    0:11:19.402490 H:m:s
    0.4284

Early stopping after 6 epochs.
lr=0.01, hidden_dim=16:
    1427 parameters
    0:03:57.227688 H:m:s
    0.4321

Early stopping after 10 epochs.
lr=0.01, hidden_dim=24:
    2523 parameters
    0:06:36.402405 H:m:s
    0.4238

Early stopping after 15 epochs.
lr=0.01, hidden_dim=32:
    3875 parameters
    0:10:18.807842 H:m:s
    0.4219

Early stopping after 14 epochs.
lr=0.01, hidden_dim=48:
    7347 parameters
    0:09:24.318881 H:m:s
    0.4240

Early stopping after 8 epochs.
lr=0.01, hidden_dim=64:
    11843 parameters
    0:05:14.845230 H:m:s
    0.4272

Early stopping after 13 epochs.
lr=0.01, hidden_dim=128:
    40067 parameters
    0:09:49.965929 H:m:s
    0.4287

Early stopping after 6 epochs.
lr=0.01, hidden_dim=256:
    145667 parameters
    0:04:15.462002 H:m:s
    0.4461

Early stopping after 27 epochs.
lr=0.001, hidden_dim=8:
    587 parameters
    0:21:30.648002 H